# Parse development notebook

### Notebook purpose
This notebook is development space for python parse.ts replacement and upgrade.
It reads specified google sheets and output actants.json file, which can be imported to inkVisitor RethinkDB.py

### Prerequisities
 * generated json schema for all used objects (run generate-json-schemas.py)

### JSon schemas for the actants
...
...

### The import tables:
 * Texts
 * Manuscripts (must be done alongside T, David thinks) = O of class defined by col. class_id
   * will have legacyId M
   * Prostě je přeparsujeme na O s metaprop "has - class - manuscript codex" (see class_id)
   *
 * Resources
 * C
 * A

### Notes
 * tables contain hyperlinks formulas, Dator was updated to handle them in resulting "label|url" format in the celss

### TODOS
#### A
 * make code for creation of entity dataset, which will generate hash id, i.e. I need do NOT make relationt through legacyId
 *
#### B
 * ..
 *
#### C
 * ..
 *


### Input variables

In [340]:
input_tables = ["texts", "manuscripts", "resources", "actions" , "concepts"]

#                   sheet_name,  code, header_in_row
input_sheets = {
    "texts" : ("Texts","13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5), #https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=2056508047
    "manuscripts" : ("Manuscripts", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "resources" : ("Resources", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "actions" :  ("Statements","1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ", 5), # https://docs.google.com/spreadsheets/d/1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ/edit#gid=0
    "concepts" : ("Concepts","1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4", 5) # https://docs.google.com/spreadsheets/d/1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4/edit#gid=0
}

table_to_entity = {
    "texts" : "ITerritory",
    "manuscripts" : "IResource",
    "resources" : "IResource",
    "actions" :  "IAction",
    "concepts" : "IConcept"
}

root_sheet_url = "https://docs.google.com/spreadsheets/d/"
google_api_dotenv_path = "../env/.env.googleapi"  # contains google api specs for sheet access with Dator
schema_path = '../schemas/' # path for dir with scheas
json_schemas = {}  # holder for schemas, so they can be used for jsonschema validate

In [341]:
# import subprocess
# subprocess.run("python generate-json-schemas.py", shell=True,capture_output=True)

### Libraries

In [342]:
import os, warlock, json
from datetime import datetime
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from copy import deepcopy
from shutil import copyfile

import uuid

def get_uuid_id():
    return str(uuid.uuid4())

# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


### Initialisation

In [343]:
load_dotenv(google_api_dotenv_path) # fills os.environ['GDRIVE_API_CREDENTIALS']
d = dator.Dator(loglevel=10, print_log_online=True, cache=True, project_name="inkvisitor-import") # expects 'GDRIVE_API_CREDENTIALS' in the global system variables (os.environ)
d.google_authenticate()
logger = d.logger

20 2022-03-04 16:24:25 : Google authentification start
20 2022-03-04 16:24:25 : Google authentification end
20 2022-03-04 16:24:25 : Dator initiation succesfull end


In [344]:
# read all schemas inside and warlock them as globally available classes
schema_filenames = os.listdir(schema_path)
json_classes = {}
for schema in schema_filenames:
    name = schema.split(".")[0]
    file_handler = open(schema_path + schema,"r")
    schema_json = json.load(file_handler)
    json_schemas[name] = schema_json
    globals()[name] = warlock.model_factory(schema_json)
    json_classes[name] = globals()[name]
    logger.info("Class " + name + " available.")

logger.info(f"There are {len(json_classes.keys())} json classes available ({' '.join(json_classes.keys())}).")

2022-03-04 16:24:25,066 INFO Class IAction available.
2022-03-04 16:24:25,069 INFO Class IConcept available.
2022-03-04 16:24:25,071 INFO Class IEntity available.
2022-03-04 16:24:25,073 INFO Class ILabel available.
2022-03-04 16:24:25,076 INFO Class IProp available.
2022-03-04 16:24:25,078 INFO Class IResource available.
2022-03-04 16:24:25,080 INFO Class IStatement available.
2022-03-04 16:24:25,082 INFO Class ITerritory available.
2022-03-04 16:24:25,084 INFO Class IUser available.
2022-03-04 16:24:25,085 INFO There are 9 json classes available (IAction IConcept IEntity ILabel IProp IResource IStatement ITerritory IUser).


In [345]:


class InkVisitorEntityFactory():

    entity_classes = json_classes

    json_class_defaults = {
        'IAction':{
            'class':'A', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'entities':{'a1':[],'a2':[],'s':[]},'valencies':{'a1':'','a2':'','s':''},'status':'1'}, 'props':[], 'notes':[]
        },
        'IConcept':{
            'class':'C', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'status':'1'}, 'props':[], 'notes':[]
        },
        'IProp':{
            'bundleEnd':False,'bundleStart':False, 'certainty':'1', 'children':[], 'elvl':'1',  'id':'', 'logic':'1', 'mood':[], 'moodvariant':'1', 'bundleOperator':'=', 'type': {'elvl':'1','id':'','logic':'1','partitivity':'1','virtuality':'1'},'value':{'elvl':'1', 'id':'', 'logic':'1', 'partitivity':'1', 'virtuality':'1'}
        },
        'IResource':{
             'class':'R', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'link':''}, 'props':[], 'notes':[]
        },
        'IStatement':{
             'class':'S', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'actants':[], 'actions':[], 'references':[],'tags':[],'territory': {'id':'','order':0}, 'text':''}, 'props':[], 'notes':[],
        },
        'ITerritory':{
            'class':'T', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'parent':{ "id": "T0", "order": 0 }}, 'props':[], 'notes':[]
        },
    }

    def __init__(self):
        pass

    def make(self,entity_name, override_object = {}):
        object = type(self).json_class_defaults[entity_name]
        object.update(override_object)
        return type(self).entity_classes[entity_name](deepcopy(object))

    def validate_defaults(self):
        for e in self.json_class_defaults:
            test = self.make(e, self.json_class_defaults[e])
            d.logger.info(f"Class {e} validated.")


IEF = InkVisitorEntityFactory()
IEF.validate_defaults()

2022-03-04 16:24:25,123 INFO Class IAction validated.
2022-03-04 16:24:25,127 INFO Class IConcept validated.
2022-03-04 16:24:25,131 INFO Class IProp validated.
2022-03-04 16:24:25,135 INFO Class IResource validated.
2022-03-04 16:24:25,142 INFO Class IStatement validated.
2022-03-04 16:24:25,147 INFO Class ITerritory validated.


In [346]:
# empty value unifier
def unify_empty_value(df: pd.DataFrame, empty_values = ['NA',"#N/A","#VALUE!"], unified_empty_value = ''):
    for naner in empty_values:
        df = df.replace(naner,unified_empty_value)
    df. fillna(unified_empty_value, inplace=True)
    return df



# load all input tables
tables = {}
header_infos = {}
for key, sheet in input_sheets.items():
    logger.info(f"Calling for {key} with sheet_name {sheet[0]}.")
    tables[key], header_infos[key] = d.load_df_from_gsheet(sheet[0],root_sheet_url + sheet[1], sheet[0], fromCache=True, header_in_row=sheet[2], clean=True, fillna=True, cleanByColumn="label", parse_hyperlink_formulas=True)
    tables[key] = unify_empty_value(tables[key])
    header_infos[key] = unify_empty_value(header_infos[key])
    tables[key]['uuid'] = tables[key].apply(lambda x: get_uuid_id(), axis=1)  # generate unique id for each row


2022-03-04 16:24:25,185 INFO Calling for texts with sheet_name Texts.


20 2022-03-04 16:24:25 : Loading dataset from pickle cache Texts
20 2022-03-04 16:24:25 : Loading separated dataset header from pickle cache Texts


2022-03-04 16:24:25,253 INFO Calling for manuscripts with sheet_name Manuscripts.
2022-03-04 16:24:25,277 INFO Calling for resources with sheet_name Resources.
2022-03-04 16:24:25,292 INFO Calling for actions with sheet_name Statements.
2022-03-04 16:24:25,356 INFO Calling for concepts with sheet_name Concepts.


20 2022-03-04 16:24:25 : Loading dataset from pickle cache Manuscripts
20 2022-03-04 16:24:25 : Loading separated dataset header from pickle cache Manuscripts
20 2022-03-04 16:24:25 : Loading dataset from pickle cache Resources
20 2022-03-04 16:24:25 : Loading separated dataset header from pickle cache Resources
20 2022-03-04 16:24:25 : Loading dataset from pickle cache Statements
20 2022-03-04 16:24:25 : Loading separated dataset header from pickle cache Statements
20 2022-03-04 16:24:25 : Loading dataset from pickle cache Concepts
20 2022-03-04 16:24:25 : Loading separated dataset header from pickle cache Concepts


In [347]:
tables['texts']

,id,label,language,label_short,text_name_original,detail,region_covered,region_covered_id,region_covered_label,microregion_covered,...,dissinet_person,number_defendants,number_persons,persons_index_link,places_index_link,old_genre_general,old_genre_label,note,parsing_rows_explained,uuid
0,T1,Process against Bernard Niort and his family,English,,,Early 1234.,Languedoc,L0015,Languedoc,,...,,,,,,,deposition,,,73d3e3e3-e224-4903-a621-a854cdc6ceef
1,T2,Sentences of William Arnold and Stephen of Sai...,English,,,,Languedoc,L0015,Languedoc,Toulousain #Lauragais,...,RS?,,,,,register,sentence,End-folio sometimes cited as 184v (e.g. Roche...,,9a1df56e-817b-4e8e-8296-996d6bf6568c
2,T3,Peter Seila’s Register of Penances,English,Seila,Penitenciae fratris Petri Sellani,Penitenciae fratris Petri Sellani. Register of...,Languedoc,L0015,Languedoc,Quercy (west),...,RS,,,,,register,sentence #culpa,,,a354f0b7-1e20-4824-a7f5-7d27b339cd4e
3,T4,Register FFF of the Carcassonne inquisition,English,FFF,,,Languedoc,L0015,Languedoc,Montségur #Lauragais #Cabardès #Quercy (east) ...,...,,,,,,register,deposition,,,4be53c9b-32c6-4749-a9aa-264736ed7536
4,T5,Confirmation of depositions before Ferrer and ...,English,,,,Languedoc,L0015,Languedoc,,...,,,,,,register,,,,e2174818-5f79-400b-9732-0646b7badc2a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,T139,letter of Evervin of Steinfeld to Bernard of C...,English,,,,,,,,...,,,,,,,,,,8d4c38af-69c9-444a-99f9-a72b6cad0242
138,T140,letter from Liège to pope,English,,,,,,,,...,,,,,,,,,,2cedf1f6-47c2-46ec-a0d0-58718fe22068
139,T141,Annales Aquenses,Latin,,,,,,,,...,,,,,,,,,,98db2c0f-e542-4de9-84c0-82fd234dbdec
140,T142,Annales Rodenses,Latin,,,,,,,,...,,,,,,,,,,9d773433-7fb6-452e-957a-979344e99089


#### Territories (general?) parsing instruction
there is info column : parsing_rows_explained, comments on the rows

row 0 : human comment
row 1 : source node for props
row 2 : prop type for props OR further details
row 3 : parse keyword (inside, discard, propvalue, special, propvalue_2nd)

possibilities by parse "keyword" (in the last row = 4)
 * inside  (two variants: A> row2==empty --> put in header_name json_field, B: row2 set -> put in row2 json_field)
 * discard
 * propvalue in row 3 +  concept id inrow  2 : classical prop relation
 * special > completely custom behavior in row 0


 * ???, ??? discard  > taken as discard
 * prop with ':' value and other inforomation > taken provisionally as discard

In [348]:
# for controlling entity and mapping of its fields
class EntityMapper():
                                    # FROM  : TO
    enum_mapper = {'language': {"English":"eng","Latin":"lat","Occitan":"oci"}}
    IFK = InkVisitorEntityFactory()

    def __init__(self, entity_type, logger = d.logger):
        self.entity =  type(self).IFK.make(entity_type)
        self.logger = logger

    # method invoker
    def update_inside_field(self, field_name, input_value):
        if input_value != '':

            if "#" in input_value or "~" in input_value:
                self.logger.info(f"ALERT # or ~ in the input value {input_value}")

            update_operation = "update_" + field_name
            update_func = getattr(self, update_operation, self.update_generic)
            update_func(field_name, input_value)
        else:
            raise Exception(f"Trying to update {field_name} with empty input value.")

    #########################################################################################################
    # the naming of procedures corresponds to the name of the input_table fields,  used for inside operations

    def update_language(self, field_name, input_value):
        if input_value in type(self).enum_mapper['language']:
            self.entity['language'] = type(self).enum_mapper['language'][input_value]
        else:
            self.entity['language'] = input_value # will raise error

    def update_note(self, field_name, input_value):
        #self.logger.info(f"Updating note with {input_value}.")
        self.entity['notes'].append(input_value)

    def update_id(self, field_name, input_value):
        #self.entity['id'] = input_value
        self.entity['id'] = get_uuid_id()


    def update_label(self, field_name, input_value):
        self.entity['label'] = "_I_"+input_value

    def update_generic(self, field_name, input_value):
        self.entity[field_name] = input_value



class EntityMapperFactory():
    def __init__(self):
        pass

    def make(self,entity_name):
        return EntityMapper(entity_name)


# CONTROL CLASS
class ParseController():
    def __init__(self, entity_list: [], keyword_row_id = 3,  logger = d.logger):
        self.entity_list = entity_list
        self.logger = logger
        self.parsers = {}

        for e in self.entity_list:
            self.parsers[e] = Parser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)

# WORKER CLASS
class Parser():
    EMP = EntityMapperFactory()

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        self.name = name
        self.logname = name.upper()
        self.input_header_df = header_df
        self.input_table_df = table_df
        self.prepared_table = pd.DataFrame()
        self.keyword_row_id =  keyword_row_id
        self.columns = self.input_header_df.columns.tolist()

        self.parsing_instruction = {}
        self.oper_columns = {'discard':[],'inside':[],'special':[],'unknown':[],'propvalue':[],'propvalue_2nd':[]}
        self.logger = logger

        # parsed json data holder
        self.js_objects = []

        # RUN
        self.process_header_instructions()
        self.prepare_input_table()

    # "parsing" instructions
    def process_header_instructions(self) -> (pd.DataFrame, pd.DataFrame):
        keyword_row = self.input_header_df.iloc[self.keyword_row_id]
        prop_type_row = self.input_header_df.iloc[self.keyword_row_id - 1]
        source_node_row = self.input_header_df.iloc[self.keyword_row_id - 2]

        log_uncertain_instructions = []

        for c in self.columns:
            instruction_candidate = str(keyword_row.at[c])
            prop_type_candidate = str(prop_type_row.at[c])
            source_node_candidate = str(source_node_row.at[c])

            if c == '':
                self.logger.error(f"{self.logname} There is empty column in the dataset.")
                raise Exception(f"{self.logname} There is empty column in the dataset.")

            if "?" in instruction_candidate or "?" in prop_type_candidate or "?" in source_node_candidate:
                log_uncertain_instructions.append(f"{c.upper()}:{instruction_candidate},{prop_type_candidate},{source_node_candidate}")
            if 'discard' in instruction_candidate:
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)
            elif 'propvalue' in instruction_candidate:
                # TODO validation
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue', 'type': prop_type, 'source':source_node}
                    self.oper_columns['propvalue'].append(c)
            elif 'special' in instruction_candidate:
                # TODO will look for custom functions registered by column name
                instruction  = {'operation':'special', 'target': None}
                self.oper_columns['special'].append(c)
            elif 'inside' in instruction_candidate:
                instruction  = {'operation':'inside', 'target': None}
                if len(prop_type_candidate) > 0:
                    instruction  = {'operation':'inside', 'target': prop_type_candidate}

                self.oper_columns['inside'].append(c)
            else:
                instruction = {'operation':'unknown', 'target': None}
                self.oper_columns['unknown'].append(c)
            self.parsing_instruction[c] = instruction

        self.logger.info(f"{self.logname} Uncertain parsing instructions in {len(log_uncertain_instructions)} columns: " + " ".join(log_uncertain_instructions) + ".")
        return self.parsing_instruction

    def prepare_input_table(self):
        ip = self.input_table_df.copy()

        # discard  columns with discard and unknown operations
        ip.drop(columns=self.oper_columns['discard']+self.oper_columns['unknown'], inplace=True)

        self.logger.info(f"{self.logname} {len(self.oper_columns['discard']+self.oper_columns['unknown'])} columns have been dropped (discard:{len(self.oper_columns['discard'])}, unknown:{len(self.oper_columns['unknown'])}). Table now has {len(ip.columns)} columns, inside:{len(self.oper_columns['inside'])},propvalue:{len(self.oper_columns['propvalue'])}, special:{len(self.oper_columns['special'])}. Originally {self.input_table_df.shape[1]} columns.")

        self.prepared_table = ip


    def prepare_property(self):
        pass


    def make_row_object(self):
        class_name = table_to_entity[self.name]
        return type(self).EMP.make(class_name)

    def parse_rows(self):
        for key, row in self.prepared_table.iterrows():
            # logger.info(f"{self.name} Processing row {key}")
            entity_mapper = self.make_row_object()
            for name, value in row.items():
                if name in self.parsing_instruction:
                    operation = self.parsing_instruction[name]
                else:
                    continue # silently ignore unknown columns
                #logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")

                if operation['operation'] == 'inside' and value != '':
                    #logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    if operation['target']:
                        name = operation['target']

                    entity_mapper.update_inside_field(name,value)

            self.js_objects.append(entity_mapper.entity)


In [349]:
#cp = ParseController(entity_list=['texts','manuscripts','resources','concepts','actions'])
cp = ParseController(entity_list=['texts'])

2022-03-04 16:24:25,649 INFO TEXTS Uncertain parsing instructions in 8 columns: TIMERELATION1_TYPE:???,, TIMERELATION1_TARGET_ID:???,, TIMERELATION2_TYPE:???,, TIMERELATION2_TARGET_ID:???,, TIMERELATION3_TYPE:???,, TIMERELATION3_TARGET_ID:???,, TIMERELATION4_TYPE:???,, TIMERELATION4_TARGET_ID:???,,.
2022-03-04 16:24:25,655 INFO TEXTS 66 columns have been dropped (discard:58, unknown:8). Table now has 40 columns, inside:8,propvalue:28, special:3. Originally 106 columns.


In [350]:
cp.parsers['texts'].parse_rows()

In [351]:
cp.parsers['texts'].js_objects

[{'class': 'T',
  'id': '245be591-88ec-4ee6-b100-6fa2de9bf795',
  'label': '_I_Process against Bernard Niort and his family',
  'language': 'eng',
  'detail': 'Early 1234.',
  'data': {'parent': {'id': 'T0', 'order': 0}},
  'props': [],
  'notes': ['Douais has a partial ed. Bruschi – Biller (eds.), Texts and the Repression of Medieval Heresy…, 13, n. 28, and Bruschi, The Wandering Heretics…, 4, refer to the ed. of the whole vol. 21 in preparation by Jörg Feuchter, but the project seems discontinued.']},
 {'class': 'T',
  'id': '50a0afe0-515d-4260-8e22-836419e311df',
  'label': '_I_Sentences of William Arnold and Stephen of Saint-Thibéry',
  'language': 'eng',
  'detail': '',
  'data': {'parent': {'id': 'T0', 'order': 0}},
  'props': [],
  'notes': ['Bruschi – Biller (eds.), Texts and the Repression of Medieval Heresy…, 13, n. 28, and Bruschi, The Wandering Heretics…, 4, refer to the ed. of the whole Doat vol. 21 in preparation by Jörg Feuchter, but this editorial project seems disconti

**Goal:**  to have a json file full of territories, which will be added to the mock up actants.json file and inserted to RethinkDb and fully visisble, accessible in inkVisitor.

Steps
 * study the mockup actants.json structure  DONE
 * prepare a code intepreting and operatinalizing David's import instruction in the header_df  DONE
 * parse the table to the territories.json
   * helper class for "territory", which fills default values
   *
 * merge json files
 * import ...

Forseen complexities
 * '#' hash notation, so called *multiples*, in the id fields, C2015#C0156. ...
 * '~' tilda notation
 * SOLVED, urls in cell (formula =HYPERLINK), SOLVED in Dator class


In [352]:
print(json.dumps(cp.parsers['texts'].js_objects[0]))

{"class": "T", "id": "245be591-88ec-4ee6-b100-6fa2de9bf795", "label": "_I_Process against Bernard Niort and his family", "language": "eng", "detail": "Early 1234.", "data": {"parent": {"id": "T0", "order": 0}}, "props": [], "notes": ["Douais has a partial ed. Bruschi \u2013 Biller (eds.), Texts and the Repression of Medieval Heresy\u2026, 13, n. 28, and Bruschi, The Wandering Heretics\u2026, 4, refer to the ed. of the whole vol. 21 in preparation by J\u00f6rg Feuchter, but the project seems discontinued."]}


Steps
 * get json string and save it as file
 * merge with existing entities.json

In [354]:
# save json object list in the text file
with open('../datasets/all-test/new_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(cp.parsers['texts'].js_objects, f)

# read  entities.json
with open('../datasets/all/entities.json','r') as f:
    #entities_content = f.readlines()
    entities_content = f.read().replace('\n', '')

# read  entities.json
with open('../datasets/all-test/new_entities.json','r') as f:
    #entities_content = f.readlines()
    new_entities_content = f.read().replace('\n', '')


# write new
with open('../datasets/all-test/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content = entities_content[0:-1] +", " + new_entities_content[1:]
    f.write(str(merge_content))